In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [ ]:
def gettingUrl(url):

    driver = webdriver.Chrome("C:\\Users\\rishu\\chromedriver")   #replace the file path by the address of chromedriver in your system
    driver.get(url)
    time.sleep(5)
    
    html = driver.page_source
    #this renders the JS code and stores all of this information in static HTMl code.
    return html

In [ ]:
#extracting data from acs nano site
html = gettingUrl("https://pubs.acs.org/doi/10.1021/acsnano.9b06394")

In [ ]:
# #to find out the details of the research paper 5
# soup = BeautifulSoup(html, 'lxml')
# paperNames = soup.find_all('span',class_ = 'NLM_article-title')
# for paper in paperNames:
#     print(paper.text)

In [ ]:
# for paper in paperNames:
#     print(paper.text)

In [ ]:
#finding out the number of times the research paper number 5 is mentioned in the text.
soup_new = BeautifulSoup(html,'lxml')  #creating an instance of the BeautifulSoup library
a_tags = soup_new.find('div', class_="article_content-left ui-resizable")
count = a_tags.find_all('a',class_="ref5")
print(len(count))

# index = 0

# for i in count:
#     data  = i.text
#     print(i)
#     dataextract = re.findall('(\(5)',data)
#     if(dataextract):
#         # print(dataextract)
#         index = index + 1

# # print(index)
        



In [ ]:
soup2 = BeautifulSoup(html,'lxml')  #creating an instance of the BeautifulSoup library
links = soup2.find('div', class_="article_content-left ui-resizable")
paperLinksgoogleScholar=links.find_all('a',class_="google-scholar")  # creating a list containing the google scholar links for all the reference papers.

In [ ]:
index=0
scholarLinks=[]
for link in paperLinksgoogleScholar:
    scholarLinks.append(link['href'])   #A new list containing the google scholar links for only the first three papers is created. 
    index=index+1

In [ ]:
for link in scholarLinks:
    print(link)

In [ ]:
# for link in paperLinks:
#     driver.get(link)
#     driver.find_element(By.XPATH, value='//*[@id="gs_res_ccl_mid"]/div/div[2]/div[3]/a[2]').click()    
    


In [45]:
html = gettingUrl(scholarLinks[4])
soup3 = BeautifulSoup(html,'lxml')  #creating an instance of the BeautifulSoup library
content = soup3.find('div', class_="gs_a")
authors = content.find_all('a')
# print(authors)
for author in authors:
    print(author.text)

B Anasori
ÛG Gogotsi


In [46]:
#finding out the publication house link of the paper from google scholar.
paperLink = soup3.find('h3',class_ = 'gs_rt')
print(paperLink.a['href'])

https://link.springer.com/content/pdf/10.1007/978-3-030-19026-2.pdf


In [44]:
driver = webdriver.Chrome("C:\\Users\\rishu\\chromedriver")
driver.get(scholarLinks[4])
# driver.find_element(By.XPATH, value='//*[@id="gs_res_ccl_mid"]/div/div[2]/div[3]/a[2]').click()
driver.find_element(By.LINK_TEXT, "Cite").click()


time.sleep(5)
# print(driver.current_url)
# try:
#     element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.CLASS_NAME, "gs_citr"))
#     )
#     element.click()
# except:
#         driver.quit()


In [ ]:
html = gettingUrl(driver.current_url)
# print(driver.current_url)
soup4 = BeautifulSoup(html, 'lxml')
paperName = soup4.find('div',class_ = 'gs_citr')
print(paperName.i.text)  #printing the name of the 5th paper.

In [ ]:
# # finding out the publication house link of the paper from google scholar.
# paperLink = soup4.find('h3',class_ = 'gs_rt')
# print(paperLink.a['href'])